In [ ]:
import serial
import time
from struct import *

In [ ]:
#------------------------------------------------------------#
# Base function to open the serial port connection
#------------------------------------------------------------#
def open_serial_port(port, baudrate, timeout):
    ser = serial.Serial()
    ser.port = port    
    ser.baudrate = baudrate
    ser.timeout = timeout
    ser.open()

    if(ser.is_open == True):
        print("=== Connection open ===")
        print("COM Port = {}".format(ser.port))
        print("Baudrate = {}".format(ser.baudrate))
        print("Data bits = {}".format(ser.bytesize))
        print("Parity = {}".format(ser.parity))
        print("Stop bits = {}".format(ser.stopbits))    
    else: 
        print("Connection not open!!!")
        
    return ser


#------------------------------------------------------------#
# Base function to create the 10-byte command string
#------------------------------------------------------------#
def encode_cmd(cmd, field1, field2):
    packed_cmd = pack('>H',cmd)
    packed_address = pack('>L', field1)
    packed_wrdata = pack('>L', field2) 
    
    cmd_str = packed_cmd + packed_address + packed_wrdata
    return cmd_str


#------------------------------------------------------------#
# Base function to execute a command and return a response
#------------------------------------------------------------#
def execute_cmd_str(cmd_str):
      
    ser.write(cmd_str)
    time.sleep(.1)

    nchars = ser.in_waiting

    if nchars != 0:
        response = ser.read(nchars)    
    else:
        response = 0
        print('read buffer empty') 
    
    return response


#------------------------------------------------------------#
# Base function to decode the response and convert it to a u32
#------------------------------------------------------------#
def decode_response(response):    
   
    response_int = int.from_bytes(response, byteorder='big') 
    return response_int


#------------------------------------------------------------#
# Group the three base functions above into a single function
#------------------------------------------------------------#
def execute_cmd(cmd, field1, field2):
    
    cmd_str = encode_cmd(cmd, field1, field2) 
    response = execute_cmd_str(cmd_str)
    response_int = decode_response(response)
    return response_int


#------------------------------------------------------------#
# Base function to close the serial port connection
#------------------------------------------------------------#
def close_serial_port(ser):    
    ser.close()
    if(ser.is_open == False):
        print("Connection closed")
    else: 
        print("Connection not closed!!!")





    
#============================================================#
# ==== COMMANDS ====
#============================================================#

# ==== MEMORY ====
#------------------------------------------------------------#
# Function to execute a 32-bit memory write
#------------------------------------------------------------#
def execute_mem_write(addr, data):
    cmd = 0x00D3
    field1 = addr
    field2 = data
    
    response = execute_cmd(cmd, field1, field2)       
    return response


#------------------------------------------------------------#
# Function to execute a 32-bit memory read
#------------------------------------------------------------#
def execute_mem_read(addr):
    cmd = 0x00D4
    field1 = addr
    field2 = 0x00000000
    
    read_data = execute_cmd(cmd, field1, field2)       
    return read_data


# ==== TEST UNKNOWN COMMAND ====
#------------------------------------------------------------#
# Function to test the response for an unknown command
#------------------------------------------------------------#
def execute_unknown_cmd():
    cmd = 0x1234
    field1 = 0x00000000
    field2 = 0x00000000
    
    response = execute_cmd(cmd, field1, field2)       
    return response





# ==== PMOD ACL ====
#------------------------------------------------------------#
# Function to execute PMOD_ACL_WRITE_BYTE
#------------------------------------------------------------#
def execute_pmodacl_write_byte(addr, data):
    cmd = 0x00E0
    field1 = addr
    field2 = data
    
    response = execute_cmd(cmd, field1, field2)       
    return response


#------------------------------------------------------------#
# Function to execute PMOD_ACL_READ_BYTE
#------------------------------------------------------------#
def execute_pmodacl_read_byte(addr):
    cmd = 0x00E1
    field1 = addr
    field2 = 0x00000000
    
    response = execute_cmd(cmd, field1, field2)       
    return response


#------------------------------------------------------------#
# Function to execute PMOD_ACL_READ_INTR_STATUS
#------------------------------------------------------------#
def execute_pmodacl_read_intr_status():
    cmd = 0x00E2
    field1 = 0x00000000
    field2 = 0x00000000
    
    response = execute_cmd(cmd, field1, field2)       
    return response


#------------------------------------------------------------#
# Function to execute PMOD_ACL_READ_XYDATA
#------------------------------------------------------------#
def execute_pmodacl_read_xydata():
    cmd = 0x00E3
    field1 = 0x00000000
    field2 = 0x00000000
    
    response = execute_cmd(cmd, field1, field2)       
    return response



#------------------------------------------------------------#
# Function to clear LED1 and LED2 on platform
#------------------------------------------------------------#
def execute_clear_leds():
    cmd = 0x00F0
    field1 = 0x00000000
    field2 = 0x00000000
    
    response = execute_cmd(cmd, field1, field2)       
    return response

In [ ]:
ser = open_serial_port('COM6', 115200, 2000)

In [ ]:
response = execute_cmd(0x00D3, 0x04000000, 0x12345678)
print("Response = 0x{0:0{1}X}".format(response, 8))

In [ ]:
response = execute_cmd(0x00D4, 0x04000000, 0x00000000)
print("Response = 0x{0:0{1}X}".format(response, 8))

In [ ]:
response = execute_cmd(0x00CF, 0x04000000, 0x00000000)
print("Response = 0x{0:0{1}X}".format(response, 8))

In [ ]:
response = execute_mem_write(0x04000004, 0xABCD1234)
print("Response = 0x{0:0{1}X}".format(response, 8))

In [ ]:
response = execute_mem_read(0x04000004)
print("Response = 0x{0:0{1}X}".format(response, 8))

In [ ]:
response = execute_clear_leds()
print("Response = 0x{0:0{1}X}".format(response, 8))

In [ ]:
addr = 0x02000014 
data = 0xaabbccdd

wr_response = execute_mem_write(addr, data)
rd_response = execute_mem_read(addr)
unk_response = execute_unknown_cmd()

print("Write response = 0x{0:0{1}X}".format(wr_response, 8))
print("Read data = 0x{0:0{1}X}".format(rd_response, 8))
print("Unknown command response = 0x{0:0{1}X}".format(unk_response, 8))

In [ ]:
addr_list = [0x02000000, 0x02000004, 0x02000008, 0x0200000C]
read_data_list = []

for i in addr_list:
    read_data = execute_mem_read(i)
    read_data_list.insert(i, read_data)
    
for i in read_data_list:
    print("Read data = 0x{0:0{1}X}".format(i, 8))    

In [ ]:
pmod_acl_reg_data = execute_pmodacl_read_byte(acl_reg_addr)
print("Read data = 0x{0:0{1}X}".format(pmod_acl_reg_data, 8))

In [ ]:
intr_status = execute_pmodacl_read_intr_status()
print("Read response = 0x{0:0{1}X}".format(intr_status, 8))

In [ ]:
xy_data = execute_pmodacl_read_xydata()
print("Write response = 0x{0:0{1}X}".format(xy_data, 8))

In [ ]:
resp = execute_clear_leds()
print("Write response = 0x{0:0{1}X}".format(resp, 8))

In [ ]:
close_serial_port(ser)